In [64]:
import copy
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

from pre_processing import *
from algorithms import *
from spectrum_gen import *
from data_gen import *
from analysis import *
from datasets import *
from evaluation import *
#cr_filtering()
#cr_read_norm()1dasdas
import h5py
import scanpy as sc
from sklearn.metrics import adjusted_rand_score
import squidpy as sq

In [ ]:
adata_enhanced = sc.read("adata_enhanced.h5ad")
adata_filtered = sc.read("adata_filtered.h5ad")
orig_adata = sc.read("orig_adata.h5ad")



In [4]:
orig_adata1 = orig_adata.copy()
orig_adata1.X = np.exp(np.array(orig_adata1.X.toarray())-1)
sc.pp.highly_variable_genes(orig_adata1,flavor='seurat_v3' , n_top_genes=1000)
orig_adata1.var.sort_values(by=['highly_variable_rank']).head(10)

/Users/jonathankarin/.conda/envs/spctral_jan/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


,MT,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,highly_variable,highly_variable_rank,means,variances,variances_norm,dispersions,dispersions_norm
Ttr,False,13980,3.586622,1.523144,73.725756,190837.0,12.159180,13312,True,0.0,0.963435,15.993514,4.341565,4.290444,0.707107
Sst,False,3119,0.174936,0.161214,94.138100,9308.0,9.138737,2999,True,1.0,0.411220,0.095008,3.704944,1.784881,5.553298
Npy,False,2571,0.109889,0.104260,95.168020,5847.0,8.673855,2500,True,2.0,0.397104,0.048820,3.046880,1.513128,3.791640
Otx2os1,False,66,0.001691,0.001690,99.875959,90.0,4.510859,66,True,3.0,0.368318,0.000649,2.579925,1.390471,3.436121
Fhl4,False,27,0.000564,0.000564,99.949256,30.0,3.433987,26,True,4.0,0.368131,0.000317,2.285383,1.230355,2.972028
Cntn5,False,937,0.032777,0.032251,98.238987,1744.0,7.464510,922,True,5.0,0.374366,0.007828,2.279974,1.187996,2.849251
Dennd1c,False,37,0.000789,0.000789,99.930462,42.0,3.761200,36,True,6.0,0.368221,0.000413,2.260644,1.189643,2.854025
Samsn1,False,40,0.000808,0.000808,99.924823,43.0,3.784190,40,True,7.0,0.368269,0.000467,2.219726,1.180628,2.827897
Ifi44,False,22,0.000451,0.000451,99.958653,24.0,3.218876,21,True,8.0,0.368151,0.000275,2.185081,1.012690,2.341133
Enpp2,False,4597,0.271576,0.240257,91.360322,14450.0,9.578519,4511,True,9.0,0.420643,0.072025,2.160064,1.311202,3.611882


In [5]:
adata_filtered1 = adata_filtered.copy()
adata_filtered1.X = np.exp(np.array(adata_filtered1.X)-1)
sc.pp.highly_variable_genes(adata_filtered1,flavor='seurat_v3' , n_top_genes=1000)
adata_filtered1.var.sort_values(by=['mean_counts']).head(10)

/Users/jonathankarin/.conda/envs/spctral_jan/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:62: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


,MT,n_cells_by_counts,mean_counts,log1p_mean_counts,pct_dropout_by_counts,total_counts,log1p_total_counts,n_cells,highly_variable,highly_variable_rank,means,variances,variances_norm,dispersions,dispersions_norm
Bach2it1,False,13,0.000263,0.000263,99.975568,14.0,2.708050,12,True,0.0,0.367935,0.000017,1.863981,-0.197473,3.490860
Gm26620,False,11,0.000263,0.000263,99.979326,14.0,2.708050,10,True,1.0,0.367931,0.000013,1.644461,-0.343544,2.645132
4930509G22Rik,False,12,0.000244,0.000244,99.977447,13.0,2.639057,12,True,2.0,0.367924,0.000011,1.613548,-0.393072,2.358375
Gm11630,False,12,0.000263,0.000263,99.977447,14.0,2.708050,12,True,3.0,0.367908,0.000007,1.564770,-0.439345,2.090463
Gm37309,False,14,0.000301,0.000301,99.973688,16.0,2.833213,14,True,4.0,0.367940,0.000015,1.544724,-0.365254,2.519439
AU040972,False,13,0.000263,0.000263,99.975568,14.0,2.708050,13,True,5.0,0.367942,0.000016,1.531477,-0.360271,2.548290
Clec4n,False,16,0.000320,0.000319,99.969929,17.0,2.890372,14,True,6.0,0.367938,0.000015,1.522811,-0.386626,2.395696
Oasl1,False,16,0.000338,0.000338,99.969929,18.0,2.944439,14,True,7.0,0.367927,0.000011,1.502007,-0.449737,2.030293
Cct6b,False,14,0.000282,0.000282,99.973688,15.0,2.772589,13,True,8.0,0.367928,0.000011,1.497146,-0.451477,2.020223
Hapln3,False,13,0.000263,0.000263,99.975568,14.0,2.708050,13,True,9.0,0.367933,0.000013,1.492174,-0.432492,2.130141


In [63]:
a_dataframe = orig_adata1.var.sort_values(by=['highly_variable_rank']).head(500)
index = a_dataframe.index
orig_data_list = list(index)

for i in orig_data_list:
    print(i)

Ttr
Sst
Npy
Otx2os1
Fhl4
Cntn5
Dennd1c
Samsn1
Ifi44
Enpp2
Gpc5
Gm2061
Gm12976
Prox2
Cd163l1
RP24-374B14.4
Mettl11b
Gm4279
Vip
Tmprss6
Elf4
Hexb
Adarb2
Gm38103
Trpm3
Gm37691
Gnrh1
RP23-458G12.1
Gm29856
Gm9754
H2-Ab1
Gm14005
Cfap53
Klhl33
Tnfrsf17
Gm16268
Myo18b
Sp110
Gm12043
Gm10033
Gm37120
Csf2rb
Tctex1d1
Hsd17b2
Rassf10
Kcnj5
Tmem220
Dlg2
Crhbp
Slc9b1
Tspan2os
Cst7
Camk1d
Crybb3
8430408G22Rik
Gm38186
Vtn
Fmo2
Ccr5
Piga
Gm12239
Gm9530
RP23-114P22.5
Gm37328
RP24-132L16.12
Iqcf3
Fam166b
1700016A09Rik
1500012K07Rik
Trim56
Uhrf1
D6Ertd474e
Cd36
Oasl1
AU040972
1700012B09Rik
Thbs4
Gm5106
Lhx5
Apobec1
BC043934
RP24-156O12.1
A330094K24Rik
Cdc25c
Cyp2j8
Cx3cr1
Cct6b
Bach2it1
Piwil1
Clcf1
Spc25
Lrrtm4
Arhgap11a
Tal1
Atp13a5
Inmt
Dock8
Gal
Olfr78
Dhx58os
Kmo
Lyve1
A630081D01Rik
1700003F12Rik
Lsp1
D930015M05Rik
Gm26666
Nrg3
Tnni2
Gm13429
P2ry1
Melk
Sgca
H2-Eb1
Atp6v1c2
4930509G22Rik
Mir6236
Ckap2l
Dmrta2
1810059C17Rik
Myh11
Ihh
Sema3b
Npas3
Gm16049
Slco2b1
Gimap4
Helz2
Gm17509
Ctss
BC030867
S1pr4


In [62]:
a_dataframe = adata_filtered1.var.sort_values(by=['highly_variable_rank']).head(500)
index = a_dataframe.index
filtered_list = list(index)

for i in filtered_list:
    print(i)

Bach2it1
Gm26620
4930509G22Rik
Gm11630
Gm37309
AU040972
Clec4n
Oasl1
Cct6b
Hapln3
C030010L15Rik
Gm29088
Gm15893
Gm38340
Scn5a
4930404I05Rik
Lif
Siglecf
Ihh
5830444F18Rik
Gm5127
RP24-423L4.3
4933433G15Rik
Pla2g2d
RP23-189N6.3
Sln
Tspan32os
Hist1h4j
A330093E20Rik
Gm37598
Cdc25c
Rs1
Gm29455
Zc3h12a
Gm16548
Gm20707
Gm28322
Gm12324
Ttc39d
A530083I20Rik
Hsd17b14
Aspm
Cyp2j8
Lhx5
RP23-120C8.7
Iqcf3
Tnni2
S1pr4
4930467D21Rik
Gm20633
Far2os2
Chaf1b
RP24-351E22.2
Fndc1
Gm28386
Gimap4
Gm13429
Ptgfr
1810013D15Rik
A330015K06Rik
Mir142hg
Il27ra
Gm4279
Dennd2c
Serpind1
D630036G22Rik
C1qtnf7
Gm12976
Clcf1
Gm26676
Gm38186
Gm29560
Ctsw
A230108P19Rik
Dcdc2c
RP23-293K21.1
Gm29629
Nespas
2810417H13Rik
Gm16168
Piwil1
Gm37935
Vmn1r4
AI847159
Inpp5d
Cst7
Gm37733
Zbp1
Gm10701
Capg
Gm15704
Ankrd61
RP24-377E5.1
Nkain1
D6Ertd474e
Kcnk5
Gm20404
Gm16069
Nr6a1os
RP24-372J15.5
E2f8
RP24-142B15.7
Ly6g6d
Gm37629
RP23-370G17.3
Gm13936
Gm12043
Hsd17b2
Foxf1
Dytn
Sbk3
Dst
Gm26829
Gm15614
Map3k7cl
Mtl5
2010013B24Rik
Prrg1


In [65]:
sq.gr.spatial_autocorr(orig_adata, mode="moran")
orig_adata.uns["moranI"].head(10)

,I,pval_norm,var_norm,pval_norm_fdr_bh
Ttr,0.695610,0.0,0.000003,0.0
Hpca,0.478294,0.0,0.000003,0.0
Mbp,0.441040,0.0,0.000003,0.0
Enpp2,0.423907,0.0,0.000003,0.0
Plp1,0.407010,0.0,0.000003,0.0
1500015O10Rik,0.403830,0.0,0.000003,0.0
Mobp,0.320221,0.0,0.000003,0.0
Ccdc153,0.290516,0.0,0.000003,0.0
Mal,0.253253,0.0,0.000003,0.0
Ptgds,0.253026,0.0,0.000003,0.0


In [66]:
sq.gr.spatial_autocorr(adata_filtered, mode="moran")
adata_filtered.uns["moranI"].head(10)

,I,pval_norm,var_norm,pval_norm_fdr_bh
Ssu2,0.042159,0.0,0.000003,0.0
Epb4.2,0.034213,0.0,0.000003,0.0
Zeb2,0.026949,0.0,0.000003,0.0
Anln,0.026662,0.0,0.000003,0.0
Ugt8a,0.025558,0.0,0.000003,0.0
Gsn,0.025497,0.0,0.000003,0.0
Dock10,0.025239,0.0,0.000003,0.0
Gkn3,0.025020,0.0,0.000003,0.0
Tmem63a,0.024866,0.0,0.000003,0.0
Tspan2,0.024465,0.0,0.000003,0.0


In [71]:
from sklearn.metrics import calinski_harabasz_score , silhouette_score

In [111]:
adata_filtered3 = adata_filtered[adata_filtered.obs['cluster'].isin(['Oligodendrocytes','Polydendrocytes'])]
calinski_harabasz_score(adata_filtered3.X,adata_filtered3.obs['cluster'])

33.91024346750053

In [80]:
orig_adata3 = orig_adata[orig_adata.obs['cluster'].isin(['Oligodendrocytes','Polydendrocytes'])]
calinski_harabasz_score(orig_adata3.X.toarray(),orig_adata3.obs['cluster'])

164.39673093318171

In [81]:
adata_enhanced3 = adata_enhanced[adata_enhanced.obs['cluster'].isin(['Oligodendrocytes','Polydendrocytes'])]
calinski_harabasz_score(adata_enhanced3.X,adata_enhanced3.obs['cluster'])

590.1369836322456

In [82]:
adata_filtered3 = adata_filtered[adata_filtered.obs['cluster'].isin(["Endothelial_Tip", "Ependymal"])]
calinski_harabasz_score(adata_filtered3.X,adata_filtered3.obs['cluster'])

11.830359289052732

In [83]:
orig_adata3 = orig_adata[orig_adata.obs['cluster'].isin(["Endothelial_Tip", "Ependymal"])]
calinski_harabasz_score(orig_adata3.X.toarray(),orig_adata3.obs['cluster'])

20.108244043097052

In [85]:
adata_enhanced3 = adata_enhanced[adata_enhanced.obs['cluster'].isin(["Endothelial_Tip", "Ependymal"])]
calinski_harabasz_score(adata_enhanced3.X,adata_enhanced3.obs['cluster'])

34.57346104153735

In [86]:
sq.gr.spatial_neighbors(orig_adata, coord_type="generic")
sq.gr.nhood_enrichment(orig_adata, cluster_key="cluster")

  0%|          | 0/1000 [00:00<?, ?/s]

In [101]:
enr_matrix = np.array((orig_adata.uns["cluster_nhood_enrichment"]['zscore']))
for i in range(enr_matrix.shape[0]):
    enr_matrix[i,i]=0
from numpy import unravel_index
unravel_index(enr_matrix.argmax(), enr_matrix.shape)

(10, 11)

In [103]:
enr_matrix[10,11]=0
unravel_index(enr_matrix.argmax(), enr_matrix.shape)

(5, 4)

In [122]:
enr_matrix[unravel_index(enr_matrix.argmax(), enr_matrix.shape)]=0
unravel_index(enr_matrix.argmax(), enr_matrix.shape)

(11, 10)

In [123]:
enr_matrix[unravel_index(enr_matrix.argmax(), enr_matrix.shape)]=0
unravel_index(enr_matrix.argmax(), enr_matrix.shape)

(4, 5)

In [124]:
enr_matrix[unravel_index(enr_matrix.argmax(), enr_matrix.shape)]=0
unravel_index(enr_matrix.argmax(), enr_matrix.shape)

(4, 3)

In [125]:
adata_filtered3 = adata_filtered[adata_filtered.obs['cluster'].isin(['Endothelial_Tip','Endothelial_Stalk'])]
calinski_harabasz_score(adata_filtered3.X,adata_filtered3.obs['cluster'])

6.050488884446616

In [126]:
orig_adata3 = orig_adata[orig_adata.obs['cluster'].isin(['Endothelial_Tip','Endothelial_Stalk'])]
calinski_harabasz_score(orig_adata3.X.toarray(),orig_adata3.obs['cluster'])

25.511776450886153

In [127]:
adata_enhanced3 = adata_enhanced[adata_enhanced.obs['cluster'].isin(['Endothelial_Tip','Endothelial_Stalk'])]
calinski_harabasz_score(adata_enhanced3.X,adata_enhanced3.obs['cluster'])

140.94733329931793

In [128]:
sq.gr.spatial_autocorr(orig_adata, mode="moran")
orig_adata.uns["moranI"].head(100)

,I,pval_norm,var_norm,pval_norm_fdr_bh
Ttr,0.708740,0.0,0.000011,0.0
Plp1,0.511602,0.0,0.000011,0.0
Hpca,0.492940,0.0,0.000011,0.0
Mbp,0.487394,0.0,0.000011,0.0
Enpp2,0.457366,0.0,0.000011,0.0
...,...,...,...,...
Aqp4,0.057504,0.0,0.000011,0.0
Klf2,0.057413,0.0,0.000011,0.0
Aspa,0.057177,0.0,0.000011,0.0
Ptprb,0.056720,0.0,0.000011,0.0


In [137]:
list_of_genes = ['Ttr', 'Sst','Npy', 'Clec4n', 'Siglecf']
for gene in list_of_genes:
    print((orig_adata.uns["moranI"]).loc[gene])

I                   0.708740
pval_norm           0.000000
var_norm            0.000011
pval_norm_fdr_bh    0.000000
Name: Ttr, dtype: float64
I                   0.356141
pval_norm           0.000000
var_norm            0.000011
pval_norm_fdr_bh    0.000000
Name: Sst, dtype: float64
I                   0.274257
pval_norm           0.000000
var_norm            0.000011
pval_norm_fdr_bh    0.000000
Name: Npy, dtype: float64
I                  -0.000211
pval_norm           0.478072
var_norm            0.000011
pval_norm_fdr_bh    0.495606
Name: Clec4n, dtype: float64
I                  -0.000170
pval_norm           0.483111
var_norm            0.000011
pval_norm_fdr_bh    0.495606
Name: Siglecf, dtype: float64


In [ ]:
(orig_adata.uns["moranI"]).loc["Sst"]